In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore")

import cv2
from skimage import io
from tqdm import tqdm
tqdm.pandas()
import re
import urllib
import requests
import selenium
from selenium import webdriver as driver
from selenium.webdriver.common.by import By

In [2]:
wd = driver.Chrome('F:/ChromeDriver/chromedriver.exe')

In [4]:
search_terms = [
    "Men Topwear Formal",
    "Men Topwear Casual",
    "Men Topwear Party",
    "Women Topwear Formal",
    "Women Topwear Casual",
    "Women Topwear Party",
    "Men Bottomwear Formal",
    "Men Bottomwear Casual",
    "Men Bottomwear Party",
    "Women Bottomwear Formal",
    "Women Bottomwear Casual",
    "Women Bottomwear Party",
    "Men Footwear Formal",
    "Men Footwear Casual",
    "Men Footwear Party",
    "Women Footwear Formal",
    "Women Footwear Casual",
    "Women Footwear Party",
]

In [5]:
product_ids, product_urls, image_urls, descriptions, brands, genders, categories, occasions = [], [], [], [], [], [], [], []
MAX_PRODUCTS_PER_CATEGORY = 1000

for search_term in search_terms:
    search_term = search_term.lower()
    query = f"https://www.myntra.com/{search_term.replace(' ', '-')}"
    total = 0
    wd.get(query)
    next_button_found = True
    while total < MAX_PRODUCTS_PER_CATEGORY:
        time.sleep(2)
        products = wd.find_elements(By.CLASS_NAME , "product-base")

        for product in products:
            try:
                image = product.find_element(By.TAG_NAME,"img")
            except:
                continue
            total += 1
            product_ids.append(f"{''.join(list(x[0] for x in search_term.split()))}_{'{0:0>10}'.format(str(total))}")
            product_urls.append(product.find_element(By.TAG_NAME , "a").get_property("href"))
            image_urls.append(image.get_property("src"))
            descriptions.append(image.get_property("alt"))
            brands.append(image.get_property("alt").split()[0])
            genders.append(search_term.split()[0])
            categories.append(search_term.split()[1])
            occasions.append(search_term.split()[2])
        try:
            wd.find_element(By.CLASS_NAME , "pagination-next").click()
        except:
            break

In [6]:
wd.close()
wd.quit()

In [7]:
df = pd.DataFrame()

df['product_id'] = product_ids
df['product_url'] = product_urls
df['image_url'] = image_urls
df['description'] = descriptions
df['brand'] = brands
df['gender'] = genders
df['category'] = categories
df['type'] = occasions

df.to_csv("./fashionProducts.csv", index=False)

In [10]:
df.head()

,product_id,product_url,image_url,description,brand,gender,category,type
0,mtf_0000000001,https://www.myntra.com/blazers/invictus/invict...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",INVICTUS Brown Slim Fit Single-Breasted Formal...,INVICTUS,men,topwear,formal
1,mtf_0000000002,https://www.myntra.com/blazers/raymond/raymond...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",Raymond Men Dark Blue Slim Fit Solid Single-Br...,Raymond,men,topwear,formal
2,mtf_0000000003,https://www.myntra.com/blazers/hangup-trend/ha...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",hangup trend Men Black Solid Bandhgala Formal ...,hangup,men,topwear,formal
3,mtf_0000000004,https://www.myntra.com/blazers/arrow/arrow-men...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",Arrow Men Olive Green Solid Slim-Fit Single-Br...,Arrow,men,topwear,formal
4,mtf_0000000005,https://www.myntra.com/blazers/raymond/raymond...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",Raymond Men Checked Single Breasted Blazer,Raymond,men,topwear,formal


In [11]:
BASE_DIR = "../images"

def downloadImage(row):
    path = row.product_id
    r = requests.request("GET", str(row.image_url))
    with open(os.path.join(BASE_DIR, path + ".jpg"), "wb") as file:
        file.write(r.content)

In [12]:
df.progress_apply(downloadImage , axis = 1)